In [1]:
import pandas as pd
from datetime import date, datetime
from datetime import datetime as dt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO
from selenium.webdriver.support.ui import Select

# Fon Fiyatları Çekimi

In [9]:
sureklicekim = pd.concat(
                [pd.read_csv("concated_mainyear_2020.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("concated_mainyear_2021.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("concated_mainyear_2022.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("concated_mainyear_2023.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("sureklicekimfonlar.csv").reset_index(drop=True).iloc[:, 1:]],
              ignore_index=True)
sureklicekim

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer
0,02.01.2020,AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,35.677757,"456,001.0",303,"16,269,093.1"
1,02.01.2020,AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,0.575406,"201,742,677.0","3,046","116,083,937.71"
2,02.01.2020,AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,0.025357,"128,719,961.0",7,"3,263,938.64"
3,02.01.2020,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,"2,868,807,200.0",11,"89,688,581.9"
4,02.01.2020,ABU,AK PORTFÖY BİREBİR BANKACILIK DEĞİŞKEN ÖZEL FON,0.019345,"612,106,546.0",117,"11,841,232.29"
...,...,...,...,...,...,...,...
879090,23.01.2024,ZVB,ATLAS PORTFÖY SERBEST (TL) FON,26.20202,"3,750,538.0",10,"98,271,670.75"
879091,23.01.2024,ZVO,ZİRAAT PORTFÖY ÜÇÜNCÜ SERBEST (TL) FON,2.621511,"222,059,759.0",12,"582,132,026.61"
879092,23.01.2024,ZYC,ZİRAAT PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,1.608178,"1,833,786,550.0",1,"2,949,055,584.75"
879093,23.01.2024,ZYD,ZİRAAT PORTFÖY BEŞİNCİ SERBEST (TL) FON,1.609395,"1,844,187,892.0",1,"2,968,026,814.41"


In [10]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("haftasonu")
    pass
else:
    if pd.to_datetime(sureklicekim['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut.")
        pass
    else:
        while True:
            driver = webdriver.Chrome()
        
            start_date = pd.to_datetime('today')
            end_date = pd.to_datetime('today')
        
            driver.get("https://www.tefas.gov.tr/TarihselVeriler.aspx")
            
            time.sleep(10)
            
            # Set start date
            driver.execute_script('document.getElementById("TextBoxStartDate").setAttribute("value", arguments[0])', start_date.strftime('%d.%m.%Y'))
        
            # Set end date
            driver.execute_script('document.getElementById("TextBoxEndDate").setAttribute("value", arguments[0])', end_date.strftime('%d.%m.%Y'))
        
            # Click the search button
            driver.find_element(By.ID, "ButtonSearchDates").click()
        
            print("Yükleniyor...")
        
            # Wait for the data to load (adjust the time.sleep value if needed)
            time.sleep(60)
        
            # Check if the table data is present
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "td.dt-center.sorting_1")))
                print("Veriler girilmiş.")
        
                print("Sayfa yapılandırılıyor...")
        
                dropdown = driver.find_element(By.NAME, "table_general_info_length")
                select = Select(dropdown)
                select.select_by_value("250")
        
                print("Sayfa / yeni sayfa çekimine başlandı...")
                df_list = []
                while True:
                    # Sayfa kaynağını çek
                    page_source = driver.page_source
                    soup = BeautifulSoup(page_source, 'html.parser')
                    table = soup.find('table', {'id': 'table_general_info'})
                    html_content = str(table)
                    df = pd.read_html(StringIO(html_content), thousands='.')[0]
                    df["Tarih"] = pd.read_html(StringIO(html_content))[0]["Tarih"]
                    # Elde edilen veriyi listeye ekle
                    df_list.append(df)
        
                    # "Sonraki" butonunun durumunu kontrol et
                    next_button = driver.find_element(By.ID, "table_general_info_next")
                    if "disabled" in next_button.get_attribute("class"):
                        break  # Eğer buton etkisizse döngüyü kır
        
                    # "Sonraki" butonuna tıkla ve bekle
                    next_button.click()
                    time.sleep(1.3)
        
                print("Sayfa / yeni sayfa çekimi tamamlandı...")
                print("Birleştiriliyor...")
                df = pd.concat(df_list, ignore_index=True)
                
                print("Formatlanıyor...")
                newdf = df.copy()
                newdf['Tarih'] = pd.to_datetime(df['Tarih'], format="%d.%m.%Y")
                newdf['Tarih'] = newdf['Tarih'].dt.strftime("%d.%m.%Y")
                newdf['Fon Kodu'] = df['Fon Kodu'].astype(str)
                newdf['Fon Adı'] = df['Fon Adı'].astype(str)
                
                newdf['Fiyat'] = pd.to_numeric(df['Fiyat'].str.replace(".", "").str.replace(",", "."))
                newdf['Fiyat'] = newdf.apply(lambda x: "{:,}".format(x['Fiyat']), axis=1)
                
                newdf['Tedavüldeki Pay Sayısı'] = pd.to_numeric(df['Tedavüldeki Pay Sayısı'].str.replace(".", "").str.replace(",", "."))
                newdf['Tedavüldeki Pay Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Tedavüldeki Pay Sayısı']), axis=1)
                
                newdf['Fon Toplam Değer'] = pd.to_numeric(df['Fon Toplam Değer'].str.replace(".", "").str.replace(",", "."))
                newdf['Fon Toplam Değer'] = newdf.apply(lambda x: "{:,}".format(x['Fon Toplam Değer']), axis=1)
                
                newdf['Kişi Sayısı'] = pd.to_numeric(df['Kişi Sayısı'])
                newdf['Kişi Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Kişi Sayısı']), axis=1)
                
                
                print("Formatlama tamamlandı.")
                sureklicekimfonlar = pd.concat([sureklicekimfonlar,newdf],ignore_index=True)
                sureklicekimfonlar.to_csv("sureklicekimfonlar.csv")
                sureklicekim = pd.concat([sureklicekim,newdf],ignore_index=True)
                df = []
                newdf = []
                sureklicekimfonlar = []
                break
                
        
            except:
                print("Veriler girilmemiş, 30 saniye sonra yeniden denenecek.")
                # Continue the loop for another attempt
                continue
        
            finally:
                
                time.sleep(30)
                driver.quit()

data mevcut.


# Altın Çekimi

In [3]:
altin = pd.read_csv("altininvesting.csv")

In [ ]:
if pd.to_datetime('today').date().weekday() == 5:  # Monday is 0, Sunday is 6
    print("ctesi")
    pass
else:
    print("sa")

In [13]:
if pd.to_datetime(df['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut.")
        pass
else:
    print("sa")

data mevcut.


In [2]:
# Selenium WebDriver'ı başlat
driver = webdriver.Chrome()  # Chrome tarayıcısı için
url = "https://tr.investing.com/currencies/gau-try-historical-data"
driver.get(url)

# Tablonun olduğu HTML kodunu çek
table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')

# BeautifulSoup kullanarak HTML'i işle
soup = BeautifulSoup(table_html, 'html.parser')

# BeautifulSoup ile tabloyu dataframe'e çevir
table = soup.find(name='table')
html_string = str(table)

# StringIO kullanarak HTML dizesini DataFrame'e çevir
df = pd.read_html(StringIO(html_string))[0]

# DataFrame'i yazdır
print(df)

# Selenium WebDriver'ı kapat
driver.quit()


         Tarih      Şimdi     Açılış     Yüksek      Düşük  Hac.  Fark %
0   23.01.2024  1.970,260  1.966,722  1.983,399  1.966,446   NaN  +0.19%
1   22.01.2024  1.966,523  1.969,516  1.973,866  1.962,361   NaN  -0.16%
2   21.01.2024  1.969,637  1.971,902  1.972,172  1.969,287   NaN   0.00%
3   19.01.2024  1.969,589  1.962,068  1.979,510  1.959,537   NaN  +0.37%
4   18.01.2024  1.962,243  1.947,682  1.963,208  1.944,090   NaN  +0.74%
5   17.01.2024  1.947,772  1.964,168  1.968,546  1.939,287   NaN  -0.84%
6   16.01.2024  1.964,239  1.987,391  1.998,388  1.958,234   NaN  -1.17%
7   15.01.2024  1.987,480  1.980,617  1.992,200  1.979,186   NaN  +0.35%
8   14.01.2024  1.980,533  1.981,991  1.982,373  1.979,627   NaN  -0.15%
9   12.01.2024  1.983,503  1.967,511  1.995,155  1.962,636   NaN  +0.82%
10  11.01.2024  1.967,421  1.945,602  1.968,102  1.939,488   NaN  +1.12%
11  10.01.2024  1.945,685  1.956,678  1.965,292  1.942,516   NaN  -0.57%
12  09.01.2024  1.956,862  1.952,532  1.963,579  1.

In [5]:
df

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,23.01.2024,"1.970,260","1.966,722","1.983,399","1.966,446",NaN,+0.19%
1,22.01.2024,"1.966,523","1.969,516","1.973,866","1.962,361",NaN,-0.16%
2,21.01.2024,"1.969,637","1.971,902","1.972,172","1.969,287",NaN,0.00%
3,19.01.2024,"1.969,589","1.962,068","1.979,510","1.959,537",NaN,+0.37%
4,18.01.2024,"1.962,243","1.947,682","1.963,208","1.944,090",NaN,+0.74%
5,17.01.2024,"1.947,772","1.964,168","1.968,546","1.939,287",NaN,-0.84%
6,16.01.2024,"1.964,239","1.987,391","1.998,388","1.958,234",NaN,-1.17%
7,15.01.2024,"1.987,480","1.980,617","1.992,200","1.979,186",NaN,+0.35%
8,14.01.2024,"1.980,533","1.981,991","1.982,373","1.979,627",NaN,-0.15%
9,12.01.2024,"1.983,503","1.967,511","1.995,155","1.962,636",NaN,+0.82%


In [23]:
pd.concat([df.iloc[:1], altin]).reset_index(drop=True)

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,23.01.2024,"1.970,260","1.966,722","1.983,399","1.966,446",NaN,+0.19%
1,23.01.2024,"1.969,874","1.966,722","1.983,399","1.966,446",NaN,"0,17%"
2,22.01.2024,"1.966,523","1.969,516","1.973,866","1.962,361",NaN,"-0,16%"
3,21.01.2024,"1.969,637","1.971,902","1.972,172","1.969,287",NaN,"0,00%"
4,19.01.2024,"1.969,589","1.962,068","1.979,510","1.959,537",NaN,"0,37%"
...,...,...,...,...,...,...,...
1267,06.01.2020,"300,177","301,206","303,438","299,500",NaN,"-0,35%"
1268,05.01.2020,"301,218","300,737","304,721","300,737",NaN,"1,09%"
1269,03.01.2020,"297,960","292,844","298,495","292,844",NaN,"1,75%"
1270,02.01.2020,"292,842","290,356","293,185","290,185",NaN,"0,86%"
